# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [20]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [21]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [22]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [23]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [24]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [25]:
#Creating a keyspace

session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkifydb
    WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)

#### Set Keyspace

In [26]:
session.set_keyspace('sparkifydb')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## **QUERY 1**
##### Give me the artist, song title, and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [27]:
## Creating table for query 1
# PRIMARY KEY: session_id - partition key \
# item_in_session - clustering key

query_1 = "CREATE TABLE IF NOT EXISTS song_information"
query_1 = query_1 + """(session_id int,item_in_session int, 
                        artist TEXT,song TEXT, length FLOAT,
                        PRIMARY KEY(session_id, item_in_session))
                    """

session.execute(query_1)
                    

In [28]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_information(session_id, item_in_session, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        ##Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [29]:
## SELECT query to give me the information about the artist, song title and song's length in the music app history \
##that was heard during sessionId = 338, and itemInSession = 4

query = """SELECT artist, length, song
            FROM song_information
            WHERE session_id = 338 AND item_in_session = 4
        """

rows = session.execute(query)

for row in rows: 
    print('Artist:{}, \n song title: {}, \n song lenght: {}'.format(row.artist, row.song, (row.length)))

Artist:Faithless, 
 song title: Music Matters (Mark Knight Dub), 
 song lenght: 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## **QUERY 2**
##### Give me the only following: name of artist, song(sorted by itemInSession) and user(first and last name) for userid = 10, sessionid = 182

In [30]:
## Creating table for query 2
# PRIMARY KEY: user_id - partition key \
# session_id, item_in_session - clustering keys

query_2 = "CREATE TABLE IF NOT EXISTS artist_information"
query_2 = query_2 + """(user_id int, session_id int, item_in_session int,
                        artist TEXT, song TEXT, 
                        user_first_name TEXT, user_last_name TEXT,
                        PRIMARY KEY((user_id, session_id), item_in_session))
                    """

session.execute(query_2)
                    

In [31]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO artist_information(user_id, session_id, item_in_session,
                                                artist, song, user_first_name, user_last_name)"""
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        ##Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [34]:
## SELECT query to give me the information about name of artist, song (sorted by itemInSession) and user (first and last name) \
## for userid = 10, sessionid = 182

query = """SELECT artist, song, user_first_name, user_last_name,item_in_session
            FROM artist_information
            WHERE user_id = 10 AND session_id = 182
        """

rows = session.execute(query)

for row in rows: 
    print('artist: {}, song title: {}, user first name: {}, user last name: {},item in session: {}'.format(row.artist, row.song, row.user_first_name, row.user_last_name, row.item_in_session))

                    

artist: Down To The Bone, song title: Keep On Keepin' On, user first name: Sylvie, user last name: Cruz,item in session: 0
artist: Three Drives, song title: Greece 2000, user first name: Sylvie, user last name: Cruz,item in session: 1
artist: Sebastien Tellier, song title: Kilometer, user first name: Sylvie, user last name: Cruz,item in session: 2
artist: Lonnie Gordon, song title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), user first name: Sylvie, user last name: Cruz,item in session: 3


## **QUERY 3**
##### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
   

In [35]:
## Creating table for query 3
# PRIMARY KEY: song- partition key \
# user_id - clustering key

query_3 = "CREATE TABLE IF NOT EXISTS user_information"
query_3 = query_3 + """(song TEXT, user_id int,
                        user_first_name TEXT, user_last_name TEXT,
                        PRIMARY KEY(song, user_id))
                    """

session.execute(query_3)

In [36]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = """INSERT INTO user_information(song, user_id, user_first_name, user_last_name)"""
        query = query + "VALUES(%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [37]:
## SELECT query to give me the information about every user name (first and last) in my music app history \
## who listened to the song 'All Hands Against His Own'

query = """SELECT user_first_name, user_last_name
            FROM user_information
            WHERE song = 'All Hands Against His Own' 
        """

rows = session.execute(query)

for row in rows: 
    print('user first name: {}, user last name: {}'.format(row.user_first_name, row.user_last_name))

user first name: Jacqueline, user last name: Lynch
user first name: Tegan, user last name: Levine
user first name: Sara, user last name: Johnson


### Drop the tables before closing out the sessions

In [38]:
## Drop the table before closing out the sessions

In [39]:
session.execute("""DROP TABLE IF EXISTS song_information""")
session.execute("""DROP TABLE IF EXISTS artist_information""")
session.execute("""DROP TABLE IF EXISTS user_information""")

### Close the session and cluster connection¶

In [40]:
session.shutdown()
cluster.shutdown()